In [1]:
from bs4 import BeautifulSoup
import requests, lxml, re, json
from datetime import datetime

In [2]:
# user-agent headers to act as a "real" user visit
headers = {
    "user-agent":
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36"
}

# search query params
params = {
    "id": "com.nintendo.zara",  # app name
    "gl": "US"                  # country
}

In [3]:
html = requests.get("https://play.google.com/store/apps/details", params=params, headers=headers, timeout=10).text

In [4]:
soup = BeautifulSoup(html, "lxml")

In [5]:
app_data = []

In [8]:
# https://regex101.com/r/DrK0ih/1
basic_app_info = json.loads(re.findall(r"<script nonce=\".*\" type=\"application/ld\+json\">(.*?)</script>",
                                       str(soup.select("script")[11]), re.DOTALL)[0])

# script[12]

In [9]:
app_name = basic_app_info["name"]
app_type = basic_app_info["@type"]
app_url = basic_app_info["url"]
app_description = basic_app_info["description"].replace("\n", "")  # replace new line character with nothing
app_category = basic_app_info["applicationCategory"]
app_operating_system = basic_app_info["operatingSystem"]
app_main_thumbnail = basic_app_info["image"]

app_content_rating = basic_app_info["contentRating"]
app_rating = round(float(basic_app_info["aggregateRating"]["ratingValue"]), 1)  # 4.287856 -> 4.3
app_reviews = basic_app_info["aggregateRating"]["ratingCount"]

app_author = basic_app_info["author"]["name"]
app_author_url = basic_app_info["author"]["url"]

In [10]:
# https://regex101.com/r/VX8E7U/1
# Follow the link to better understand what regular expression is matching.
app_images_data = re.findall(r",\[\d{3,4},\d{3,4}\],.*?(https.*?)\"", str(soup.select("script")))

In [11]:
app_images = [item for item in app_images_data if app_images_data.count(item) == 1]

In [12]:
pp_images_data = re.findall(r",\[\d{3,4},\d{3,4}\],.*?(https.*?)\"", str(soup.select("script")))

app_images_not_filtered = [item for item in app_images_data]
app_images_filtered = [item for item in app_images_data if app_images_data.count(item) == 1]

print(len(app_images_not_filtered))
print(app_images_not_filtered)

print(len(app_images_filtered))
print(app_images_filtered)

'''
71
['https://play-lh.googleusercontent.com/dcv6Z-pr3MsSvxYh_UiwvJem8fktDUsvvkPREnPaHYienbhT31bZ2nUqHqGpM1jdal8', 'https://play-lh.googleusercontent.com/SVYZCU-xg-nvaBeJ-rz6rHSSDp20AK-5AQPfYwI38nV8hPzFHEqIgFpc3LET-Dmu-Q', 'https://play-lh.googleusercontent.com/Nne-dalTl8DJ9iius5oOLmFe-4DnvZocgf92l8LTV0ldr9JVQ2BgeW_Bbjb5nkVngrQ', 'https://play-lh.googleusercontent.com/yIqljB_Jph_T_ITmVFTpmDV0LKXVHWmsyLOVyEuSjL2794nAhTBaoeZDpTZZLahyRsE', 'https://play-lh.googleusercontent.com/5HdGRlNsBvHTNLo-vIsmRLR8Tr9degRfFtungX59APFaz8OwxTnR_gnHOkHfAjhLse7e', 'https://play-lh.googleusercontent.com/bPhRpYiSMGKwO9jkjJk1raR7cJjMgPcUFeHyTg_I8rM7_6GYIO9bQm6xRcS4Q2qr6mRx', 'https://play-lh.googleusercontent.com/7DOCBRsIE5KncQ0AzSA9nSnnBh0u0u804NAgux992BhJllLKGNXkMbVFWH5pwRwHUg', 'https://play-lh.googleusercontent.com/PCaFxQba_CvC2pi2N9Wuu814srQOUmrW42mh-ZPCbk_xSDw3ubBX7vOQeY6qh3Id3YE', 'https://play-lh.googleusercontent.com/fQne-6_Le-sWScYDSRL9QdG-I2hWxMbe2QbDOzEsyu3xbEsAb_f5raRrc6GUNAHBoQ', 'https://play-lh.googleusercontent.com/ql7LENlEZaTq2NaPuB-esEPDXM2hs1knlLa2rWOI3uNuQ77hnC1lLKNJrZi9XKZFb4I', 'https://play-lh.googleusercontent.com/UIHgekhfttfNCkd5qCJNaz2_hPn67fOkv40_5rDjf5xot-QhsDCo2AInl9036huUtCwf', 'https://play-lh.googleusercontent.com/7iH7-GjfS_8JOoO7Q33JhOMnFMK-O8k7jP0MUI75mYALK0kQsMsHpHtIJidBZR46sfU', 'https://play-lh.googleusercontent.com/czt-uL-Xx4fUgzj_JbNA--RJ3xsXtjAxMK7Q_wFZdoMM6nL_g-4S5bxxX3Di3QTCwgw', 'https://play-lh.googleusercontent.com/e5HMIP0FW9MCoAEGYzji9JsrvyovpZ3StHiIANughp3dovUxdv_eHiYT5bMz38bowOI', 'https://play-lh.googleusercontent.com/nv2BP1glvMWX11mHC8GWlh_UPa096_DFOKwLZW4DlQQsrek55pY2lHr29tGwf2FEXHM', 'https://play-lh.googleusercontent.com/xwWDr_Ib6dcOr0H0OTZkHupwSrpBoNFM6AXNzNO27_RpX_BRoZtKIULKEkigX8ETOKI', 'https://play-lh.googleusercontent.com/AxHkW996UZvDE21HTkGtQPU8JiQLzNxp7yLoQiSCN29Y54kZYvf9aWoR6EzAlnoACQ', 'https://play-lh.googleusercontent.com/xFouF73v1_c5kS-mnvQdhKwl_6v3oEaLebsZ2inlJqIeF2eenXjUrUPJsjSdeAd41w', 'https://play-lh.googleusercontent.com/a1pta2nnq6f_b9uV0adiD9Z1VVQrxSfX315fIQqgKDcy8Ji0BRC1H7z8iGnvZZaeg80', 'https://play-lh.googleusercontent.com/SDAFLzC8i4skDJ2EcsEkXidcAJCql5YCZI76eQB15fVaD0j-ojxyxea00klquLVtNAw', 'https://play-lh.googleusercontent.com/H7BcVUoygPu8f7oIs2dm7g5_vVt9N9878f-rGd0ACd-muaDEOK2774okryFfsXv9FaI', 'https://play-lh.googleusercontent.com/5LIMaa7WTNy34bzdFhBETa2MRj7mFJZWb8gCn_uyxQkUvFx_uOFCeQjcK16c6WpBA3E', 'https://play-lh.googleusercontent.com/iTZtyWYr4T-slu1nifgRqEhtMLmxcNagc2rDAyiWntDQWCVLlGR7rDvx0uK6z-zLujwv', 'https://play-lh.googleusercontent.com/iTZtyWYr4T-slu1nifgRqEhtMLmxcNagc2rDAyiWntDQWCVLlGR7rDvx0uK6z-zLujwv', 'https://play-lh.googleusercontent.com/EbEX3AN4FC4pu3lsElAHCiksluOVU8OgkgtWC43-wmm_aHVq2D65FmEM97bPexilUAvlAY5_4ARH8Tb3RxQ', 'https://play-lh.googleusercontent.com/_re6mcALPaqotePA0WkgYeOQ6TighHRUS62FRmREPEhyZPdGM3QmRjcSpiMt6Pz1O-WZyvEIy4mtGHj9zw', 'https://play-lh.googleusercontent.com/SI2XoFyY35xzlMz3cZdTH7SSxMDfJTJjKNtbso33YIyYknmxJnBLrfLPJ131gz3O259sB9gP9dcmSvRNFw', 'https://play-lh.googleusercontent.com/SI2XoFyY35xzlMz3cZdTH7SSxMDfJTJjKNtbso33YIyYknmxJnBLrfLPJ131gz3O259sB9gP9dcmSvRNFw', 'https://play-lh.googleusercontent.com/pzvdI66OFjncahvxJN714Tu5pHUJ_nJK--vg0tv5cpgaGNvjfwsxC-SKxoQh9_n_wEcCdSQF9FeuZeI', 'https://play-lh.googleusercontent.com/EbEX3AN4FC4pu3lsElAHCiksluOVU8OgkgtWC43-wmm_aHVq2D65FmEM97bPexilUAvlAY5_4ARH8Tb3RxQ', 'https://play-lh.googleusercontent.com/_re6mcALPaqotePA0WkgYeOQ6TighHRUS62FRmREPEhyZPdGM3QmRjcSpiMt6Pz1O-WZyvEIy4mtGHj9zw', 'https://play-lh.googleusercontent.com/_re6mcALPaqotePA0WkgYeOQ6TighHRUS62FRmREPEhyZPdGM3QmRjcSpiMt6Pz1O-WZyvEIy4mtGHj9zw', 'https://play-lh.googleusercontent.com/z6aS2wnyp16KA9CFEep7HvZd2DmwRfoR9NWm9oHWRw-tuXLE_CPbnb1OL39-a456EgA', 'https://play-lh.googleusercontent.com/z6aS2wnyp16KA9CFEep7HvZd2DmwRfoR9NWm9oHWRw-tuXLE_CPbnb1OL39-a456EgA', 'https://play-lh.googleusercontent.com/z6aS2wnyp16KA9CFEep7HvZd2DmwRfoR9NWm9oHWRw-tuXLE_CPbnb1OL39-a456EgA', 'https://play-lh.googleusercontent.com/kMnXlmzr3b8Tbzs_xDy3vq12fnZ6PM4LVlxPlFMKf_VZVkk1v7xeAUpJxW6iYab9m_w', 'https://play-lh.googleusercontent.com/kMnXlmzr3b8Tbzs_xDy3vq12fnZ6PM4LVlxPlFMKf_VZVkk1v7xeAUpJxW6iYab9m_w', 'https://play-lh.googleusercontent.com/kMnXlmzr3b8Tbzs_xDy3vq12fnZ6PM4LVlxPlFMKf_VZVkk1v7xeAUpJxW6iYab9m_w', 'https://play-lh.googleusercontent.com/5BFo6FvdAn0c10xiDKO_GZMtHmn-4qxHTtF6rarC162rCNqnA7jub30CYWmzC_DZ1l4', 'https://play-lh.googleusercontent.com/5BFo6FvdAn0c10xiDKO_GZMtHmn-4qxHTtF6rarC162rCNqnA7jub30CYWmzC_DZ1l4', 'https://play-lh.googleusercontent.com/5BFo6FvdAn0c10xiDKO_GZMtHmn-4qxHTtF6rarC162rCNqnA7jub30CYWmzC_DZ1l4', 'https://play-lh.googleusercontent.com/9EYrR6ilBWJFLt_LE_QniHZjdYlG9on6PzTOqR9tBf3SKiWU4lIDOXq-kXrrSKyyEg', 'https://play-lh.googleusercontent.com/9EYrR6ilBWJFLt_LE_QniHZjdYlG9on6PzTOqR9tBf3SKiWU4lIDOXq-kXrrSKyyEg', 'https://play-lh.googleusercontent.com/9EYrR6ilBWJFLt_LE_QniHZjdYlG9on6PzTOqR9tBf3SKiWU4lIDOXq-kXrrSKyyEg', 'https://play-lh.googleusercontent.com/pyJ4VUsm75cc800LalWMZupRMG7o-JictgTeSIUKni2Hn2ncR4m22hgV_LhTahsRt0U', 'https://play-lh.googleusercontent.com/pyJ4VUsm75cc800LalWMZupRMG7o-JictgTeSIUKni2Hn2ncR4m22hgV_LhTahsRt0U', 'https://play-lh.googleusercontent.com/pyJ4VUsm75cc800LalWMZupRMG7o-JictgTeSIUKni2Hn2ncR4m22hgV_LhTahsRt0U', 'https://play-lh.googleusercontent.com/z6aS2wnyp16KA9CFEep7HvZd2DmwRfoR9NWm9oHWRw-tuXLE_CPbnb1OL39-a456EgA', 'https://play-lh.googleusercontent.com/z6aS2wnyp16KA9CFEep7HvZd2DmwRfoR9NWm9oHWRw-tuXLE_CPbnb1OL39-a456EgA', 'https://play-lh.googleusercontent.com/z6aS2wnyp16KA9CFEep7HvZd2DmwRfoR9NWm9oHWRw-tuXLE_CPbnb1OL39-a456EgA', 'https://play-lh.googleusercontent.com/H8AjtJR4LviiM3M8dg1BrS7_XzHBziG91Cn-udo8w44fRPo5mwj6NL683JBJQslpnZOY', 'https://play-lh.googleusercontent.com/H8AjtJR4LviiM3M8dg1BrS7_XzHBziG91Cn-udo8w44fRPo5mwj6NL683JBJQslpnZOY', 'https://play-lh.googleusercontent.com/H8AjtJR4LviiM3M8dg1BrS7_XzHBziG91Cn-udo8w44fRPo5mwj6NL683JBJQslpnZOY', 'https://play-lh.googleusercontent.com/DGP09C5sfjxaawTV0JUIFTDKJ0579kmss59AkjHzvz6ry6FSjTzjHGO8GiB3BwglPI5g', 'https://play-lh.googleusercontent.com/DGP09C5sfjxaawTV0JUIFTDKJ0579kmss59AkjHzvz6ry6FSjTzjHGO8GiB3BwglPI5g', 'https://play-lh.googleusercontent.com/DGP09C5sfjxaawTV0JUIFTDKJ0579kmss59AkjHzvz6ry6FSjTzjHGO8GiB3BwglPI5g', 'https://play-lh.googleusercontent.com/tUB0fBVdN0pLn_wpfKRETC3jcraaYc7nFEDCOFsE7SUK0WCKpUWO0k3pOi-x-bPkIAo', 'https://play-lh.googleusercontent.com/tUB0fBVdN0pLn_wpfKRETC3jcraaYc7nFEDCOFsE7SUK0WCKpUWO0k3pOi-x-bPkIAo', 'https://play-lh.googleusercontent.com/tUB0fBVdN0pLn_wpfKRETC3jcraaYc7nFEDCOFsE7SUK0WCKpUWO0k3pOi-x-bPkIAo', 'https://play-lh.googleusercontent.com/z6aS2wnyp16KA9CFEep7HvZd2DmwRfoR9NWm9oHWRw-tuXLE_CPbnb1OL39-a456EgA', 'https://play-lh.googleusercontent.com/z6aS2wnyp16KA9CFEep7HvZd2DmwRfoR9NWm9oHWRw-tuXLE_CPbnb1OL39-a456EgA', 'https://play-lh.googleusercontent.com/z6aS2wnyp16KA9CFEep7HvZd2DmwRfoR9NWm9oHWRw-tuXLE_CPbnb1OL39-a456EgA', 'https://play-lh.googleusercontent.com/H8AjtJR4LviiM3M8dg1BrS7_XzHBziG91Cn-udo8w44fRPo5mwj6NL683JBJQslpnZOY', 'https://play-lh.googleusercontent.com/H8AjtJR4LviiM3M8dg1BrS7_XzHBziG91Cn-udo8w44fRPo5mwj6NL683JBJQslpnZOY', 'https://play-lh.googleusercontent.com/H8AjtJR4LviiM3M8dg1BrS7_XzHBziG91Cn-udo8w44fRPo5mwj6NL683JBJQslpnZOY', 'https://play-lh.googleusercontent.com/DGP09C5sfjxaawTV0JUIFTDKJ0579kmss59AkjHzvz6ry6FSjTzjHGO8GiB3BwglPI5g', 'https://play-lh.googleusercontent.com/DGP09C5sfjxaawTV0JUIFTDKJ0579kmss59AkjHzvz6ry6FSjTzjHGO8GiB3BwglPI5g', 'https://play-lh.googleusercontent.com/DGP09C5sfjxaawTV0JUIFTDKJ0579kmss59AkjHzvz6ry6FSjTzjHGO8GiB3BwglPI5g', 'https://play-lh.googleusercontent.com/tUB0fBVdN0pLn_wpfKRETC3jcraaYc7nFEDCOFsE7SUK0WCKpUWO0k3pOi-x-bPkIAo', 'https://play-lh.googleusercontent.com/tUB0fBVdN0pLn_wpfKRETC3jcraaYc7nFEDCOFsE7SUK0WCKpUWO0k3pOi-x-bPkIAo', 'https://play-lh.googleusercontent.com/tUB0fBVdN0pLn_wpfKRETC3jcraaYc7nFEDCOFsE7SUK0WCKpUWO0k3pOi-x-bPkIAo']
23
['https://play-lh.googleusercontent.com/dcv6Z-pr3MsSvxYh_UiwvJem8fktDUsvvkPREnPaHYienbhT31bZ2nUqHqGpM1jdal8', 'https://play-lh.googleusercontent.com/SVYZCU-xg-nvaBeJ-rz6rHSSDp20AK-5AQPfYwI38nV8hPzFHEqIgFpc3LET-Dmu-Q', 'https://play-lh.googleusercontent.com/Nne-dalTl8DJ9iius5oOLmFe-4DnvZocgf92l8LTV0ldr9JVQ2BgeW_Bbjb5nkVngrQ', 'https://play-lh.googleusercontent.com/yIqljB_Jph_T_ITmVFTpmDV0LKXVHWmsyLOVyEuSjL2794nAhTBaoeZDpTZZLahyRsE', 'https://play-lh.googleusercontent.com/5HdGRlNsBvHTNLo-vIsmRLR8Tr9degRfFtungX59APFaz8OwxTnR_gnHOkHfAjhLse7e', 'https://play-lh.googleusercontent.com/bPhRpYiSMGKwO9jkjJk1raR7cJjMgPcUFeHyTg_I8rM7_6GYIO9bQm6xRcS4Q2qr6mRx', 'https://play-lh.googleusercontent.com/7DOCBRsIE5KncQ0AzSA9nSnnBh0u0u804NAgux992BhJllLKGNXkMbVFWH5pwRwHUg', 'https://play-lh.googleusercontent.com/PCaFxQba_CvC2pi2N9Wuu814srQOUmrW42mh-ZPCbk_xSDw3ubBX7vOQeY6qh3Id3YE', 'https://play-lh.googleusercontent.com/fQne-6_Le-sWScYDSRL9QdG-I2hWxMbe2QbDOzEsyu3xbEsAb_f5raRrc6GUNAHBoQ', 'https://play-lh.googleusercontent.com/ql7LENlEZaTq2NaPuB-esEPDXM2hs1knlLa2rWOI3uNuQ77hnC1lLKNJrZi9XKZFb4I', 'https://play-lh.googleusercontent.com/UIHgekhfttfNCkd5qCJNaz2_hPn67fOkv40_5rDjf5xot-QhsDCo2AInl9036huUtCwf', 'https://play-lh.googleusercontent.com/7iH7-GjfS_8JOoO7Q33JhOMnFMK-O8k7jP0MUI75mYALK0kQsMsHpHtIJidBZR46sfU', 'https://play-lh.googleusercontent.com/czt-uL-Xx4fUgzj_JbNA--RJ3xsXtjAxMK7Q_wFZdoMM6nL_g-4S5bxxX3Di3QTCwgw', 'https://play-lh.googleusercontent.com/e5HMIP0FW9MCoAEGYzji9JsrvyovpZ3StHiIANughp3dovUxdv_eHiYT5bMz38bowOI', 'https://play-lh.googleusercontent.com/nv2BP1glvMWX11mHC8GWlh_UPa096_DFOKwLZW4DlQQsrek55pY2lHr29tGwf2FEXHM', 'https://play-lh.googleusercontent.com/xwWDr_Ib6dcOr0H0OTZkHupwSrpBoNFM6AXNzNO27_RpX_BRoZtKIULKEkigX8ETOKI', 'https://play-lh.googleusercontent.com/AxHkW996UZvDE21HTkGtQPU8JiQLzNxp7yLoQiSCN29Y54kZYvf9aWoR6EzAlnoACQ', 'https://play-lh.googleusercontent.com/xFouF73v1_c5kS-mnvQdhKwl_6v3oEaLebsZ2inlJqIeF2eenXjUrUPJsjSdeAd41w', 'https://play-lh.googleusercontent.com/a1pta2nnq6f_b9uV0adiD9Z1VVQrxSfX315fIQqgKDcy8Ji0BRC1H7z8iGnvZZaeg80', 'https://play-lh.googleusercontent.com/SDAFLzC8i4skDJ2EcsEkXidcAJCql5YCZI76eQB15fVaD0j-ojxyxea00klquLVtNAw', 'https://play-lh.googleusercontent.com/H7BcVUoygPu8f7oIs2dm7g5_vVt9N9878f-rGd0ACd-muaDEOK2774okryFfsXv9FaI', 'https://play-lh.googleusercontent.com/5LIMaa7WTNy34bzdFhBETa2MRj7mFJZWb8gCn_uyxQkUvFx_uOFCeQjcK16c6WpBA3E', 'https://play-lh.googleusercontent.com/pzvdI66OFjncahvxJN714Tu5pHUJ_nJK--vg0tv5cpgaGNvjfwsxC-SKxoQh9_n_wEcCdSQF9FeuZeI']
'''

263
['https://play-lh.googleusercontent.com/IciOnDFecb5Xt50Q2jlcNC0LPI7LEGxNojroo-s3AozcyS-vDCwtq4fn7u3wZmRna8OewG9PBrWC-i7i', 'https://play-lh.googleusercontent.com/OBVqgRK7eerY0GPfK8AOzitu5oE9ecC6kG4kURTCb1K41gpqVsN0WjmJwJh-wX8vILzpcc1kYHt56aLN2g', 'https://play-lh.googleusercontent.com/dcv6Z-pr3MsSvxYh_UiwvJem8fktDUsvvkPREnPaHYienbhT31bZ2nUqHqGpM1jdal8', 'https://play-lh.googleusercontent.com/SVYZCU-xg-nvaBeJ-rz6rHSSDp20AK-5AQPfYwI38nV8hPzFHEqIgFpc3LET-Dmu-Q', 'https://play-lh.googleusercontent.com/Nne-dalTl8DJ9iius5oOLmFe-4DnvZocgf92l8LTV0ldr9JVQ2BgeW_Bbjb5nkVngrQ', 'https://play-lh.googleusercontent.com/yIqljB_Jph_T_ITmVFTpmDV0LKXVHWmsyLOVyEuSjL2794nAhTBaoeZDpTZZLahyRsE', 'https://play-lh.googleusercontent.com/5HdGRlNsBvHTNLo-vIsmRLR8Tr9degRfFtungX59APFaz8OwxTnR_gnHOkHfAjhLse7e', 'https://play-lh.googleusercontent.com/bPhRpYiSMGKwO9jkjJk1raR7cJjMgPcUFeHyTg_I8rM7_6GYIO9bQm6xRcS4Q2qr6mRx', 'https://play-lh.googleusercontent.com/7DOCBRsIE5KncQ0AzSA9nSnnBh0u0u804NAgux992BhJllLKGNXkMbV

"\n71\n['https://play-lh.googleusercontent.com/dcv6Z-pr3MsSvxYh_UiwvJem8fktDUsvvkPREnPaHYienbhT31bZ2nUqHqGpM1jdal8', 'https://play-lh.googleusercontent.com/SVYZCU-xg-nvaBeJ-rz6rHSSDp20AK-5AQPfYwI38nV8hPzFHEqIgFpc3LET-Dmu-Q', 'https://play-lh.googleusercontent.com/Nne-dalTl8DJ9iius5oOLmFe-4DnvZocgf92l8LTV0ldr9JVQ2BgeW_Bbjb5nkVngrQ', 'https://play-lh.googleusercontent.com/yIqljB_Jph_T_ITmVFTpmDV0LKXVHWmsyLOVyEuSjL2794nAhTBaoeZDpTZZLahyRsE', 'https://play-lh.googleusercontent.com/5HdGRlNsBvHTNLo-vIsmRLR8Tr9degRfFtungX59APFaz8OwxTnR_gnHOkHfAjhLse7e', 'https://play-lh.googleusercontent.com/bPhRpYiSMGKwO9jkjJk1raR7cJjMgPcUFeHyTg_I8rM7_6GYIO9bQm6xRcS4Q2qr6mRx', 'https://play-lh.googleusercontent.com/7DOCBRsIE5KncQ0AzSA9nSnnBh0u0u804NAgux992BhJllLKGNXkMbVFWH5pwRwHUg', 'https://play-lh.googleusercontent.com/PCaFxQba_CvC2pi2N9Wuu814srQOUmrW42mh-ZPCbk_xSDw3ubBX7vOQeY6qh3Id3YE', 'https://play-lh.googleusercontent.com/fQne-6_Le-sWScYDSRL9QdG-I2hWxMbe2QbDOzEsyu3xbEsAb_f5raRrc6GUNAHBoQ', 'https://pla

In [13]:
app_user_comments = []

In [14]:
# https://regex101.com/r/SrP5DS/1
# # Follow the link to better understand what regular expression is matching.
app_user_reviews_data = re.findall(r"(\[\"gp.*?);</script>",
                                   str(soup.select("script")), re.DOTALL)

In [46]:
# Follow the links to better understand what regular expressions are matching.

for review in app_user_reviews_data:
    # https://regex101.com/r/M24tiM/1
    user_name = re.findall(r"\"gp:.*?\",\s?\[\"(.*?)\",", str(review))
    
    # https://regex101.com/r/TGgR45/1
    user_avatar = [avatar.replace('"', "") for avatar in re.findall(r"\"gp:.*?\"(https.*?\")", str(review))]

    # replace single/double quotes at the start/end of a string
    # https://regex101.com/r/iHPOrI/1
    user_comment = [comment.replace('"', "").replace("'", "") for comment in
                    re.findall(r"gp:.*?https:.*?]]],\s?\d+?,.*?,\s?(.*?),\s?\[\d+,", str(review))]

    # comment utc timestamp
    # use datetime.utcfromtimestamp(int(date)).date() to have only a date: 2022-01-09
    user_comment_date = [str(datetime.utcfromtimestamp(int(date))) for date in re.findall(r"\[(\d+),", str(review))]

    # https://regex101.com/r/GrbH9A/1
    user_comment_id = [ids.replace('"', "") for ids in re.findall(r"\[\"(gp.*?),", str(review))]
    
    # https://regex101.com/r/jRaaQg/1
    user_comment_likes = re.findall(r",?\d+\],?(\d+),?", str(review))
    
    # https://regex101.com/r/Z7vFqa/1
    user_comment_app_rating = re.findall(r"\"gp.*?https.*?\],(.*?)?,", str(review))
    for name, avatar, comment, date, comment_id, likes, user_app_rating in zip(user_name, user_avatar, user_comment, user_comment_date, user_comment_id, user_comment_likes, user_comment_app_rating):
            app_user_comments.append({
            "user_name": name,
            "user_avatar": avatar,
            "comment": comment,
            "user_app_rating": user_app_rating,
            "user__comment_likes": likes,
            "user_comment_published_at": date,
            "user_comment_id": comment_id
            })
            app_data.append({
            "app_name": app_name,
            "app_type": app_type,
            "app_url": app_url,
            "app_main_thumbnail": app_main_thumbnail,
            "app_description": app_description,
            "app_content_rating": app_content_rating,
            "app_category": app_category,
            "app_operating_system": app_operating_system,
            "app_rating": app_rating,
            "app_reviews": app_reviews,
            "app_author": app_author,
            "app_author_url": app_author_url,
            "app_screenshots": app_images
            })
        return {"app_data": app_data, "app_user_comments": app_user_comments}

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 52)

In [21]:
app_user_comments

[]

IndentationError: unexpected indent (<ipython-input-28-6b8e995f97b6>, line 16)

In [23]:
app_data

[{'app_name': 'Super Mario Run',
  'app_type': 'SoftwareApplication',
  'app_url': 'https://play.google.com/store/apps/details/Super_Mario_Run?id=com.nintendo.zara&hl=en_US&gl=US',
  'app_main_thumbnail': 'https://play-lh.googleusercontent.com/5LIMaa7WTNy34bzdFhBETa2MRj7mFJZWb8gCn_uyxQkUvFx_uOFCeQjcK16c6WpBA3E',
  'app_description': 'Control Mario with just a tap!',
  'app_content_rating': 'Everyone',
  'app_category': 'GAME_ACTION',
  'app_operating_system': 'ANDROID',
  'app_rating': 4.0,
  'app_reviews': '1637594',
  'app_author': 'Nintendo Co., Ltd.',
  'app_author_url': 'https://supermariorun.com/',
  'app_screenshots': ['https://play-lh.googleusercontent.com/IciOnDFecb5Xt50Q2jlcNC0LPI7LEGxNojroo-s3AozcyS-vDCwtq4fn7u3wZmRna8OewG9PBrWC-i7i',
   'https://play-lh.googleusercontent.com/OBVqgRK7eerY0GPfK8AOzitu5oE9ecC6kG4kURTCb1K41gpqVsN0WjmJwJh-wX8vILzpcc1kYHt56aLN2g',
   'https://play-lh.googleusercontent.com/dcv6Z-pr3MsSvxYh_UiwvJem8fktDUsvvkPREnPaHYienbhT31bZ2nUqHqGpM1jdal8',
   'h

In [24]:
len(app_data)

1

SyntaxError: 'return' outside function (<ipython-input-26-9d61a819c93f>, line 1)